In [ ]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#https://huggingface.co/ckiplab/gpt2-base-chinese
model_name = "ckiplab/gpt2-base-chinese"

# 檢查是否有 GPU 可以用（如果有會快很多）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用裝置：{device}")

使用裝置：cuda


In [ ]:
# 載入指定的語言模型與 tokenizer
# model_name 是你指定的模型名稱，例如 "ckiplab/gpt2-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 用於將文字轉換為模型輸入的 token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # 載入因果語言模型並指定裝置（例如 CPU 或 GPU）

# 定義一個函式，用來根據提示詞（prompt）與選擇的風格生成文字
def generate_text(prompt, style="default", max_new_tokens=150, temperature=0.9, top_k=50, top_p=0.95):
    # 定義可選的風格標籤字典，對應不同的語氣或文體
    style_labels = {
        "default": "一般",
        "fairy_tale": "童話",
        "ancient_chinese": "古文",
        "child_style": "兒童",
        "mystery": "懸疑",
        "sci_fi": "科幻",
        "wuxia": "武俠",
        "diary": "日記",
        "news": "新聞",
        "poetic": "詩詞"
    }

    # 根據 style 參數取得對應的中文風格標籤，若未指定則預設為「一般」
    style_tag = style_labels.get(style, "一般")

    # 將風格標籤加入提示詞中，以引導模型產生指定風格的內容
    styled_prompt = f"[風格: {style_tag}]\n{prompt}"

    # 將 styled_prompt 編碼成模型可接受的 token ID，並轉成 tensor
    input_ids = tokenizer.encode(styled_prompt, return_tensors="pt").to(device)

    # 建立 attention mask，避免模型注意到 padding 的位置（這裡預設沒有 padding，但為保險起見仍處理）
    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    # 使用模型產生文字
    output_ids = model.generate(
        input_ids,                      # 輸入的 token
        attention_mask=attention_mask, # 注意力遮罩，讓模型知道哪些位置是有效輸入
        max_new_tokens=max_new_tokens, # 生成的最大 token 數量
        temperature=temperature,       # 溫度值控制隨機性，越高越多樣化
        top_k=top_k,                   # top-k 篩選（僅在前 k 個最可能的詞中選擇）
        top_p=top_p,                   # top-p (nucleus sampling)，保留機率總和為 p 的詞集合中選擇
        do_sample=True,                # 啟用隨機取樣（非貪婪解碼）
        pad_token_id=tokenizer.eos_token_id  # 設定 pad token 為結束符號，以避免警告
    )

    # 解碼模型輸出的 token ID，轉回可讀文字，並移除多餘空白
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).replace(" ", "")
    return output_text  # 回傳最終生成的文字

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/421M [00:00<?, ?B/s]

In [ ]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "fairy_tale"
# 呼叫前面定義的 generate_text 函式，傳入提示文字與風格，產生對應風格的續寫文字
result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： fairy_tale
產生的故事： [風格:童話]故事開始於一個神秘的森林，小狐狸出現在月光下。個角色是一個大自然小子，但卻有一些玩家認為它與同樣的同伴有關。而在小狐狸的故事中，同伴有著一個名叫的少年的小狐狸。一個小狐狸的父母為大狐狸。同伴則是一個非常特殊的少年。同伴有一個大狐狸。同伴是一個小狐狸。在這個故事中，這個小狐狸是一個大狐狸，而是一個大狐狸。小狐狸在小狐狸的故事中有著一個小


In [ ]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "sci_fi"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： sci_fi
產生的故事： [風格:科幻]故事開始於一個神秘的森林，小狐狸出現在月光下。們為了一塊寶石，從小狐狸身上放置物品，用來對抗一個從小狐狸和一個在地球上看見的小狐狸的隊友。與這一部電影的拍攝在2011年12月開始，並在12月17日在美國上映。《》在第52屆電影節中獲得了多項獎項，包括奧斯卡最佳男主角。《》的導演是他的弟弟，一位好萊塢製片人。他最著名的電影作品包括：oftheof


In [ ]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "news"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： news
產生的故事： [風格:新聞]故事開始於一個神秘的森林，小狐狸出現在月光下。個小島，他與老虎一起玩耍，而老虎也在其中找到小狐狸。雖然小狐狸的人生地被關在大海中，但是他們被捕捉並受到很大的威脅，並且小狐狸大約在一個小島上被人發現。然而，一個小島的人類卻受到了不少的威脅。小狐狸不想再玩耍，反而會被強迫開船。他們將他們送回大海中，之後他們再度被捕捉，而其他的孩童則被送回大海中。在


In [ ]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "wuxia"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： wuxia
產生的故事： [風格:武俠]故事開始於一個神秘的森林，小狐狸出現在月光下。後，蝙蝠俠們變成自己的小獸人，發現自己的魔法讓蝙蝠俠很快地開始尋找一個魔法。蝙蝠俠不僅對其他人而言是很大的改變，還在其他人的故事中展現了蝙蝠俠的魅力。小狐狸大部分時間都從其他人的故事中復活。蝙蝠俠們認為魔法能夠成為他的武器。但是大部分情節在不同的角度上有不同的差異。而大部分情節在於他們的故事中都存在
